In [1]:
import pandas as pd

In [2]:
df_lesson = pd.read_csv('lessons.txt', sep='|', skiprows=[1])
df_participants = pd.read_csv('participants.txt', sep='|', skiprows=[1])
df_quality = pd.read_csv('quality.txt', sep='|', skiprows=[1])
df_users = pd.read_csv('users.txt', sep='|', skiprows=[1])

In [3]:
df_lesson.columns = [name.strip() for name in df_lesson.columns.values.tolist()]
df_lesson['scheduled_time'] = pd.to_datetime(df_lesson['scheduled_time'])
df_lesson.dropna(inplace=True, subset=['event_id'])

df_lesson['id'] = df_lesson['id'].astype('str')
df_lesson['id'] = df_lesson['id'].str.strip()

df_lesson['subject'] = df_lesson['subject'].str.strip()
df_lesson['subject'] = df_lesson['subject'].astype('category')

df_lesson['event_id'] = df_lesson['event_id'].astype(int)

df_lesson = df_lesson[df_lesson['subject'] == 'phys']


In [4]:
df_participants.columns = [name.strip() for name in df_participants.columns.values.tolist()]
df_participants.dropna(inplace=True)

df_participants['event_id'] = df_participants['event_id'].astype('int')

df_participants['user_id'] = df_participants['user_id'].astype('str')
df_participants['user_id'] = df_participants['user_id'].str.strip()


In [5]:
df_quality.columns = [name.strip() for name in df_quality.columns.values.tolist()]
df_quality['lesson_id'] = df_quality['lesson_id'].astype('str')
df_quality['lesson_id'] = df_quality['lesson_id'].str.strip()

In [6]:
df_users.columns = [name.strip() for name in df_users.columns.values.tolist()]
df_users.dropna(subset=['role'],inplace=True)
df_users['role'] = df_users['role'].astype('category')
df_users['id'] = df_users['id'].astype('str')
df_users['id'] = df_users['id'].str.strip()
df_users.drop_duplicates(subset ="id", inplace = True) 


,id,role
7,30a19496-bdaf-461c-abbc-2709ae520201,tutor


In [7]:
df_lesson = df_lesson.merge(df_participants,left_on='event_id', right_on='event_id')

#00fc6685-f53a-49bb-b960-5e0042fd3852
df_lesson[df_lesson['id'] == '00fc6685-f53a-49bb-b960-5e0042fd3852']

,id,event_id,subject,scheduled_time,user_id
8,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00,4ab072ed-8a41-4ed6-9877-8f1d1fbb24d2
9,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00,30a19496-bdaf-461c-abbc-2709ae520201


In [8]:
df_lesson = df_lesson.merge(df_quality,left_on='id', right_on='lesson_id').drop(columns=['id'])
df_lesson[df_lesson['lesson_id'] == '00fc6685-f53a-49bb-b960-5e0042fd3852']

,event_id,subject,scheduled_time,user_id,lesson_id,tech_quality
4,51341,phys,2020-01-17 12:00:00,4ab072ed-8a41-4ed6-9877-8f1d1fbb24d2,00fc6685-f53a-49bb-b960-5e0042fd3852,5
5,51341,phys,2020-01-17 12:00:00,4ab072ed-8a41-4ed6-9877-8f1d1fbb24d2,00fc6685-f53a-49bb-b960-5e0042fd3852,5
6,51341,phys,2020-01-17 12:00:00,30a19496-bdaf-461c-abbc-2709ae520201,00fc6685-f53a-49bb-b960-5e0042fd3852,5
7,51341,phys,2020-01-17 12:00:00,30a19496-bdaf-461c-abbc-2709ae520201,00fc6685-f53a-49bb-b960-5e0042fd3852,5


In [9]:
df = df_lesson.merge(df_users,left_on='user_id', right_on='id',how='left')
df[df['lesson_id'] == '00fc6685-f53a-49bb-b960-5e0042fd3852']

,event_id,subject,scheduled_time,user_id,lesson_id,tech_quality,id,role
4,51341,phys,2020-01-17 12:00:00,4ab072ed-8a41-4ed6-9877-8f1d1fbb24d2,00fc6685-f53a-49bb-b960-5e0042fd3852,5,4ab072ed-8a41-4ed6-9877-8f1d1fbb24d2,pupil
5,51341,phys,2020-01-17 12:00:00,4ab072ed-8a41-4ed6-9877-8f1d1fbb24d2,00fc6685-f53a-49bb-b960-5e0042fd3852,5,4ab072ed-8a41-4ed6-9877-8f1d1fbb24d2,pupil
6,51341,phys,2020-01-17 12:00:00,30a19496-bdaf-461c-abbc-2709ae520201,00fc6685-f53a-49bb-b960-5e0042fd3852,5,30a19496-bdaf-461c-abbc-2709ae520201,tutor
7,51341,phys,2020-01-17 12:00:00,30a19496-bdaf-461c-abbc-2709ae520201,00fc6685-f53a-49bb-b960-5e0042fd3852,5,30a19496-bdaf-461c-abbc-2709ae520201,tutor


In [10]:
del(df_lesson)
del(df_participants)
del(df_users)

In [11]:
df[df['lesson_id'] == '00fc6685-f53a-49bb-b960-5e0042fd3852']

,event_id,subject,scheduled_time,user_id,lesson_id,tech_quality,id,role
4,51341,phys,2020-01-17 12:00:00,4ab072ed-8a41-4ed6-9877-8f1d1fbb24d2,00fc6685-f53a-49bb-b960-5e0042fd3852,5,4ab072ed-8a41-4ed6-9877-8f1d1fbb24d2,pupil
5,51341,phys,2020-01-17 12:00:00,4ab072ed-8a41-4ed6-9877-8f1d1fbb24d2,00fc6685-f53a-49bb-b960-5e0042fd3852,5,4ab072ed-8a41-4ed6-9877-8f1d1fbb24d2,pupil
6,51341,phys,2020-01-17 12:00:00,30a19496-bdaf-461c-abbc-2709ae520201,00fc6685-f53a-49bb-b960-5e0042fd3852,5,30a19496-bdaf-461c-abbc-2709ae520201,tutor
7,51341,phys,2020-01-17 12:00:00,30a19496-bdaf-461c-abbc-2709ae520201,00fc6685-f53a-49bb-b960-5e0042fd3852,5,30a19496-bdaf-461c-abbc-2709ae520201,tutor


In [15]:
df.drop_duplicates(subset =['lesson_id','user_id'], inplace = True)

In [16]:
df[df['lesson_id'] == '00fc6685-f53a-49bb-b960-5e0042fd3852']

,event_id,subject,scheduled_time,user_id,lesson_id,tech_quality,id,role
4,51341,phys,2020-01-17 12:00:00,4ab072ed-8a41-4ed6-9877-8f1d1fbb24d2,00fc6685-f53a-49bb-b960-5e0042fd3852,5,4ab072ed-8a41-4ed6-9877-8f1d1fbb24d2,pupil
6,51341,phys,2020-01-17 12:00:00,30a19496-bdaf-461c-abbc-2709ae520201,00fc6685-f53a-49bb-b960-5e0042fd3852,5,30a19496-bdaf-461c-abbc-2709ae520201,tutor
